In [3]:
import json
from typing import Any

from dynamiq.connections import E2B
from dynamiq.nodes.agents.orchestrators.graph import END, START, GraphOrchestrator
from dynamiq.nodes.agents.orchestrators.graph_manager import GraphAgentManager
from dynamiq.nodes.tools.e2b_sandbox import E2BInterpreterTool
from dynamiq.nodes.types import InferenceMode
from dynamiq.prompts import Message, Prompt
from dynamiq.runnables import RunnableStatus
from llm_setup import setup_llm

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_observation" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [4]:
llm = setup_llm()
connection_e2b = E2B()

tool_code = E2BInterpreterTool(connection=connection_e2b)
llm = setup_llm(model_provider="gpt", model_name="gpt-4o-mini", temperature=0)

def code_llm(messages, structured_output=True):
    code_sample = {
        "type": "json_schema",
        "json_schema": {
            "strict": True,
            "name": "generate_code_solution",
            "schema": {
                "type": "object",
                "required": ["libraries", "code"],
                "properties": {
                    "libraries": {
                        "type": "string",
                        "description": (
                            "Libraries that have to be installed (coma separated)." " Example: 'pandas,numpy'"
                        ),
                    },
                    "code": {"type": "string", "description": "Code solution to the problem."},
                },
                "additionalProperties": False,
            },
        },
    }

    llm_result = llm.run(
        input_data={},
        prompt=Prompt(
            messages=messages,
        ),
        schema=code_sample if structured_output else None,
        inference_mode=InferenceMode.STRUCTURED_OUTPUT if structured_output else InferenceMode.XML,
    ).output["content"]

    return json.loads(llm_result) if structured_output else llm_result

def generate_code_solution(context: dict[str, Any]):
    """
    Generate a code solution
    """

    print("#####CODE GENERATION#####")

    messages = context.get("messages")

    if context.get("reiterate", False):
        messages += [Message(role="user", content="Generate code again taking into account errors. {}")]

    code_solution = code_llm(messages)
    context["solution"] = code_solution

    context["messages"] += [
        Message(
            role="assistant",
            content=f"\n Imports: {code_solution.get('libraries')} \n Code: {code_solution.get('code')}",
        )
    ]

    context["iterations_num"] += 1
    return code_solution

def reflect(context: dict[str, Any]):
    print("#####REFLECTING ON ERRORS#####")
    reflections = code_llm(messages=context.get("messages"))
    context["messages"] += [Message(role="assistant", content=f"Here are reflections on the error: {reflections}")]
    return reflections

def validate_code(context: dict[str, Any], **_):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("#####CHECKING#####")
    solution = context["solution"]

    result = tool_code.run(input_data={"python": solution.get("code"), "packages": solution.get("libraries")})
    if result.status == RunnableStatus.SUCCESS:
        print("#####SUCCESSFUL#####")
        successful_message = [
            Message(role="user", content=f"Your code executed successfully {result.output['content']}")
        ]
        context["messages"] += successful_message
        context["reiterate"] = False
    else:
        print("#####FAILED#####")
        error_message = [
            Message(
                role="user",
                content=(
                    f"Your solution failed the code execution test: {result.output['content']}."
                    " Reflect on possible errors."
                ),
            )
        ]
        context["messages"] += error_message
        context["reiterate"] = True

    return result.output.get("content")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydantic/_internal/_config.py:284: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
2024-11-11 13:43:17 - WARNING - Python-dotenv could not parse statement starting at line 2
2024-11-11 13:43:17 - WARNING - Python-dotenv could not parse statement starting at line 5
2024-11-11 13:43:17 - WARNING - Python-dotenv could not parse statement starting at line 8
2024-11-11 13:43:17 - WARNING - Python-dotenv could not parse statement starting at line 9
2024-11-11 13:43:17 - WARNING - Python-dotenv could not parse statement starting at line 11
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/litellm/utils.py:17: DeprecationWarning: 'imghdr' is deprecated 

In [5]:

orchestrator = GraphOrchestrator(
    name="Graph orchestrator",
    manager=GraphAgentManager(llm=llm),
    objective="Provide final code that succeed and reflection.",
)

orchestrator.add_node("generate_code", [generate_code_solution])
orchestrator.add_node("validate_code", [validate_code])
orchestrator.add_node("reflect", [reflect])

orchestrator.add_edge(START, "generate_code")
orchestrator.add_edge("generate_code", "validate_code")
orchestrator.add_edge("reflect", "generate_code")

orchestrator.add_conditional_edge(
    "validate_code", ["generate_code", END], lambda context: "reflect" if context["reiterate"] else END
)


In [6]:
result = orchestrator.run(
    input_data={
        "messages": [Message(role="user", content="Make 100 lines of code")],
        "iterations_num": 0,
        "reiterate": False,
    },
    config=None,
)

print("Result:")
print(result.output.get("content"))

2024-11-11 13:43:19 - INFO - Node Graph orchestrator - 8bbcec0c-42db-4d36-95f7-582de760a6be: execution started.
2024-11-11 13:43:19 - INFO - GraphOrchestrator 8bbcec0c-42db-4d36-95f7-582de760a6be: Next action: START
2024-11-11 13:43:19 - INFO - GraphOrchestrator 8bbcec0c-42db-4d36-95f7-582de760a6be: Next action: generate_code
2024-11-11 13:43:19 - INFO - Node generate_code_solution - 63adc1ed-0e2f-48ec-b65f-05c704b5f771: execution started.
2024-11-11 13:43:19 - INFO - Node OpenAI LLM - ee8cb750-88de-437f-98ff-6c17b7d0eae0: execution started.


#####CODE GENERATION#####


2024-11-11 13:43:29 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-11 13:43:29 - INFO - Node OpenAI LLM - ee8cb750-88de-437f-98ff-6c17b7d0eae0: execution succeeded in 9.8s.
2024-11-11 13:43:29 - INFO - Node generate_code_solution - 63adc1ed-0e2f-48ec-b65f-05c704b5f771: execution succeeded in 9.8s.
2024-11-11 13:43:29 - INFO - GraphOrchestrator 8bbcec0c-42db-4d36-95f7-582de760a6be: Next action: validate_code
2024-11-11 13:43:29 - INFO - Node validate_code - 98a63c92-19ea-4924-8c46-2f60551c4c16: execution started.
2024-11-11 13:43:29 - INFO - Node code-interpreter_e2b - aee50597-dca3-4fe2-8248-0d18863783eb: execution started.
2024-11-11 13:43:29 - INFO - Starting process: pip install -qq numpy pandas


#####CHECKING#####


2024-11-11 13:43:29 - INFO - Started process (id: Pl9QYDNI0LYQ)
2024-11-11 13:43:43 - INFO - Handling process exit (id: Pl9QYDNI0LYQ)
2024-11-11 13:43:43 - INFO - Starting process: python3 /home/user/7fcf7e6767d52fe5f8a5c128c4efdcb0f75536edf503ecf090837f74a5c7063e.py
2024-11-11 13:43:43 - INFO - Started process (id: o3RSsYtC5j8Q)
2024-11-11 13:43:44 - INFO - Handling process exit (id: o3RSsYtC5j8Q)
2024-11-11 13:43:44 - ERROR - Node code-interpreter_e2b - aee50597-dca3-4fe2-8248-0d18863783eb: execution error: Error during Python code execution: Traceback (most recent call last):
  File "/home/user/7fcf7e6767d52fe5f8a5c128c4efdcb0f75536edf503ecf090837f74a5c7063e.py", line 76, in <module>
    filled_df = loaded_df.fillna(loaded_df.mean())
                                 ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/pandas/core/frame.py", line 11693, in mean
    result = super().mean(axis, skipna, numeric_only, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

...

#####CHECKING#####


2024-11-11 13:44:31 - INFO - Started process (id: coUC8JKHXb9e)
2024-11-11 13:44:33 - INFO - Handling process exit (id: coUC8JKHXb9e)
2024-11-11 13:44:33 - INFO - Starting process: python3 /home/user/f3c5bd1b8ff77266ecd47e5865eec9644ae57d80c4c2930ec2c0dfaa65d8f897.py
2024-11-11 13:44:33 - INFO - Started process (id: Gthx8fMq5DWr)
2024-11-11 13:44:34 - INFO - Handling process exit (id: Gthx8fMq5DWr)
2024-11-11 13:44:34 - INFO - Node code-interpreter_e2b - aee50597-dca3-4fe2-8248-0d18863783eb: execution succeeded in 3.3s.
2024-11-11 13:44:34 - INFO - Node validate_code - 98a63c92-19ea-4924-8c46-2f60551c4c16: execution succeeded in 3.3s.
2024-11-11 13:44:34 - INFO - Node <lambda> - d8c91ed9-083e-457d-8472-24b5e2bcd0e3: execution started.
2024-11-11 13:44:34 - INFO - Node <lambda> - d8c91ed9-083e-457d-8472-24b5e2bcd0e3: execution succeeded in 0ms.
2024-11-11 13:44:34 - INFO - GraphOrchestrator 8bbcec0c-42db-4d36-95f7-582de760a6be: Next action: END
2024-11-11 13:44:34 - INFO - Node Graph Ma

#####SUCCESSFUL#####


2024-11-11 13:44:53 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-11 13:44:53 - INFO - Node OpenAI LLM - ee8cb750-88de-437f-98ff-6c17b7d0eae0: execution succeeded in 18.9s.
2024-11-11 13:44:53 - INFO - Node Graph Manager - 17895939-07fc-4a50-8cc0-c67ff421d1f4: execution succeeded in 18.9s.
2024-11-11 13:44:53 - INFO - Node Graph orchestrator - 8bbcec0c-42db-4d36-95f7-582de760a6be: execution succeeded in 1.6m.


Result:
### Final Report on Data Processing and Analysis

#### Overview
This report summarizes the successful execution of a data processing and analysis task using Python libraries, specifically `numpy`, `pandas`, `matplotlib`, and `seaborn`. The task involved generating a random dataset, performing various statistical analyses, visualizing the data, and saving the results to CSV files.

#### Libraries Used
- **Numpy**: For numerical operations and random data generation.
- **Pandas**: For data manipulation and analysis.
- **Matplotlib**: For creating static, animated, and interactive visualizations.
- **Seaborn**: For statistical data visualization.

#### Code Execution Summary
The following steps were executed in the code:

1. **Data Generation**:
   - A random dataset with 100 rows and 5 features was generated using `numpy`.
   - A DataFrame was created from this random data.

2. **Initial DataFrame Display**:
   - The first few rows of the DataFrame were printed to verify the data